In [3]:
# %matplotlib notebook
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
# plt.rcParams["font.family"] = "Times New Roman"
import seaborn as sns
sns.set_style("whitegrid")

from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

import numexpr as ne
# from astropy.table import Table
# tbdata = Table.read('datafile', format='fits')
# df = tbdata.to_pandas()

# df['class'] = df['class'].str.decode('utf-8') 

# Plot
# fig = plt.figure(figsize=(5,5))
# ax1 = fig.add_subplot(1,1,1)
# fontsize=16

# ax1.set_xlabel(r"",fontsize=fontsize)
# ax1.set_ylabel(r"",fontsize=fontsize)
# ax1.set_title(r"",fontsize=fontsize)
# ax1.tick_params(axis='both', labelsize=fontsize)
# plt.tight_layout()

In [29]:
narrow_lines = {
    
        "NA_H_BETA"	   :{"center":4862.691, 
                         "amp":"free", 
                         "disp":"NA_OIII_5007_DISP", 
                         "voff":"free",
                         "line_type":"na",
                         "label":r"H$\beta$"
                        },
        "NA_OIII_4960" :{"center":4960.295, 
                         "amp":"(NA_OIII_5007_AMP/2.98)", 
                         "disp":"NA_OIII_5007_DISP", 
                         "voff":"NA_OIII_5007_VOFF",
                         "line_type":"na",
                         "label":r"[O III]"
                        },
        "NA_OIII_5007" :{"center":5008.240, 
                         "amp":"free", 
                         "disp":"free", 
                         "voff":"free",
                         "line_type":"na",
                         "label":r"[O III]"
                        },

}

param_dict = {
    "NA_H_BETA_AMP":5.0,
    "NA_H_BETA_VOFF":2.0,
    "NA_OIII_5007_AMP":10.0,
    "NA_OIII_5007_DISP":100.0,
    "NA_OIII_5007_VOFF":0.0,
}


line_list = {**narrow_lines}
print(line_list)

{'NA_H_BETA': {'center': 4862.691, 'amp': 'free', 'disp': 'NA_OIII_5007_DISP', 'voff': 'free', 'line_type': 'na', 'label': 'H$\\beta$'}, 'NA_OIII_4960': {'center': 4960.295, 'amp': '(NA_OIII_5007_AMP/2.98)', 'disp': 'NA_OIII_5007_DISP', 'voff': 'NA_OIII_5007_VOFF', 'line_type': 'na', 'label': '[O III]'}, 'NA_OIII_5007': {'center': 5008.24, 'amp': 'free', 'disp': 'free', 'voff': 'free', 'line_type': 'na', 'label': '[O III]'}}


In [69]:
for line in list(line_list):
        for hpar in line_list[line]:
            if (line_list[line][hpar]!="free") and (hpar in ["amp","disp","voff","h3","h4","h5","h6","h7","h8","h9","h10","shape"]):
                if (isinstance(line_list[line][hpar],(int,float))):
                    line_list[line][hpar] = float(line_list[line][hpar])
                    print(line,hpar,float(line_list[line][hpar]))
#                     pass
                else:
                    print(line,hpar,ne.evaluate(line_list[line][hpar], local_dict = param_dict).item())
                        

NA_H_BETA disp 100.0
NA_OIII_4960 amp 3.3557046979865772
NA_OIII_4960 disp 100.0
NA_OIII_4960 voff 0.0


In [84]:
# Add additional components before hard and soft constraints are checked.
ncomp = 2
ncomp_list = {}
if ncomp>1:    
    for n in np.arange(ncomp-1)+2:
#         print(n)
        # Create a dictionary for each additional set of components
        ncomp_list["NCOMP_%d" % n] = {}
        for line in line_list: # line is the "parent" line
            print(line_list[line])
#             print(line)
            # Create a new child line based on the parent line
            ncomp_list["NCOMP_%d" % n][line+"_%d" % n] = {}
            for hpar in line_list[line]:
                # First non-fittable hyperparameters (center, line_type, line_profile)
                if hpar=="center":
                    ncomp_list["NCOMP_%d" % n][line+"_%d" % n][hpar] = line_list[line]["center"]
                if hpar=="line_type":
                    ncomp_list["NCOMP_%d" % n][line+"_%d" % n][hpar] = line_list[line]["line_type"]
                if hpar=="line_profile":
                    ncomp_list["NCOMP_%d" % n][line+"_%d" % n][hpar] = line_list[line]["line_profile"]
                # Now fittable hyperparameters (amp, disp, voff, h3, shape, etc.)
                # Parameters that are free in the parent will become free in the child. 
                # Parameters that are tied in the parenter will become tied to their respective child component parameters.
                if (hpar in ["amp","disp","voff","h3","h4","h5","h6","h7","h8","h9","h10","shape"]) and (line_list[line][hpar]=="free"):
                    ncomp_list["NCOMP_%d" % n][line+"_%d" % n][hpar] = line_list[line][hpar]
                elif (hpar in ["amp","disp","voff","h3","h4","h5","h6","h7","h8","h9","h10","shape"]) and (line_list[line][hpar]!="free"):
#                     print(hpar,line_list[line][hpar])
#                     print(line_list.keys())
                    for key in line_list.keys():
                        if key in line_list[line][hpar]:
                            new_hpar = line_list[line][hpar].replace(key,key+"_%d" % n)
                            ncomp_list["NCOMP_%d" % n][line+"_%d" % n][hpar] = new_hpar
                
                    
                
            
        
if ncomp_list:
    for n in ncomp_list:
        print(n)
        for line in ncomp_list[n]:
            print("\t",line)
            for hpar in ncomp_list[n][line]:
                print("\t\t",hpar,"=",ncomp_list[n][line][hpar])
        

{'center': 4862.691, 'amp': 'free', 'disp': 'NA_OIII_5007_DISP', 'voff': 'free', 'line_type': 'na', 'label': 'H$\\beta$'}
{'center': 4960.295, 'amp': '(NA_OIII_5007_AMP/2.98)', 'disp': 'NA_OIII_5007_DISP', 'voff': 'NA_OIII_5007_VOFF', 'line_type': 'na', 'label': '[O III]'}
{'center': 5008.24, 'amp': 'free', 'disp': 'free', 'voff': 'free', 'line_type': 'na', 'label': '[O III]'}
NCOMP_2
	 NA_H_BETA_2
		 center = 4862.691
		 amp = free
		 disp = NA_OIII_5007_2_DISP
		 voff = free
		 line_type = na
	 NA_OIII_4960_2
		 center = 4960.295
		 amp = (NA_OIII_5007_2_AMP/2.98)
		 disp = NA_OIII_5007_2_DISP
		 voff = NA_OIII_5007_2_VOFF
		 line_type = na
	 NA_OIII_5007_2
		 center = 5008.24
		 amp = free
		 disp = free
		 voff = free
		 line_type = na


In [87]:
amp = (0,)
print(len(amp))

1
